# trying to sort the entire Luminex dataset
+ ### there have been some previous runs 
+ ### then, there were three runs in 2021:
    * #### 20211021
        + naming only in "Multiple Data"
        + doublet (complete) data in "Rawdata.csv"
    * #### 20211102 (standard curve from 20211021)
        + naming only in "Multiple Data"
        + doublet (without standard) in Rawdata.csv
    * #### 20211222 (naming and doublets in Rawdata.xlsx)
+ ### I have received data (Multiple_Data_3-Plex_Conan_ISA.xlsx etc)
+ ### Sté has received other data later (3-Plex_ISA_conc.xlsx etc)
+ ### strategy is to
    * #### first get in all the raw data as well - assignment - value
    * #### calculate the standard curves
    * #### derive the concentrations from the standard curves
    * #### create a relational table assigning date and well to sample
   

## load in the raw data

In [ ]:
home = os.environ['HOME']
data_path = os.path.join(home, "/Users/martinszyska/Dropbox/Icke/Work/Luminex/LuminexData")

### read header info

In [ ]:
def read_csv_header(csv_file):
    '''
    reads the plate_info from a csv raw data file
    returns series with info
    '''
    
    plate_info = pd.read_csv(csv_file, nrows=6, names=['info'], sep="\t", encoding = "ISO-8859-1")
    plate_info['PlateID'] = plate_info['info'].str.split(": ").str[0]
    plate_info['data'] = plate_info['info'].str.split(": ").str[1].str.rstrip(";")
    plate_info = plate_info.drop('info', axis=1).set_index('PlateID')
    return plate_info['data']


def read_excel_header(excel_file):
    '''
    reads the plate_info from a csv raw data file
    returns series with info
    '''
    
    info = pd.read_excel(excel_file, nrows=6, header=None)
    plate_info = pd.DataFrame()
    plate_info['PlateID'] = info[0].str.split(": ").str[0]
    plate_info['data'] = info[0].str.split(": ").str[1].str.rstrip(";")
    plate_info = plate_info.set_index('PlateID')
    
    return plate_info['data']


def read_header(file, is_excel=False):
    '''
    reads all the info from a luminex header (excel or csv)
    '''

    # read basic data based on extension
    plate_info = read_excel_header(file) if is_excel else read_csv_header(file)
        
    # wrangle the data and add run and plex from file name
    plate_df = plate_info.rename({
        "Plate ID": "orgPlateID",
        "File Name": "RawdataPath",
        "Acquisition Date": "AcquisitionTime",
        "Reader Serial Number": "ReaderID", 
        "RP1 PMT (Volts)": "RP1_PMT",
        "RP1 Target": "RP1_Target"
    }).to_frame().T.reset_index(drop="True")
    
    return plate_df


#### header readout is saved as a plate-based row in the plate_df

In [ ]:
csv_file = os.path.join(data_path, "3-Plex/20211102_3-Plex_ISA_Rawdata.csv")
excel_file = os.path.join(data_path, "38-Plex/20211222_38-Plex_ISA_Rawdata.xlsx")

plate_df = read_header(csv_file)
plate_df

### read the raw data

In [ ]:
def get_run_plex(file):
    '''
    retrieve run and flex from the file name
    '''
    s = os.path.basename(file).split("_")
    run = s[0]
    plex = [d for d in s if d.endswith("Plex")][0]
    return run, plex


def read_raw_plate(file):
    '''
    reads a Luminex raw data file
    autodetects format
    returns plate_data as series and well_raw data a df
    '''
    
    # get info from filename
    run, plex = get_run_plex(file)
    
    ### header
    # read file depending on extension 
    is_excel = file.split(".")[-1].startswith("xls")
    
    # read header and add 
    header = read_header(file, is_excel)
    header['Run'] = run
    header['Plex'] = plex
    header = header.loc[:, ['Run', 'Plex'] + list(header.columns[:-2])]
    # read the raw data body
    data = pd.read_excel(file, skiprows=7) if is_excel else pd.read_csv(file, skiprows=7, sep=";", encoding = "ISO-8859-1")
    data = data.rename({'Sampling Errors':'SamplingErrors'}, axis=1)

    # adjust the Gene headers
    # get the genes and headers into the col_df
    col_df = pd.DataFrame(data.columns[2:-1])[0].str.extract(r"([^(]+) \(([0-9]+)\)").rename({0:"Gene", 1:"col"}, axis=1)
    cols = col_df.columns
    col_df['Run'] = run
    col_df['Plex'] = plex
    col_df = col_df.loc[:, ['Run', 'Plex'] + list(cols)]
    
    
    # apply the cleaned gene names to the column names
    data.columns = list(data.columns[:2]) + list(col_df['Gene']) + list(data.columns[-1:])
    # stack the data
    data = data.melt(id_vars=['Well', 'Type', 'SamplingErrors'], var_name="Gene", value_name="FI")
    data.loc[:, 'FI'] = data['FI'].str.replace(",", ".").str.replace(r"***", "0", regex=False).astype(float)
    cols = data.columns
    # add run as id
    data['Run'] = run
    data['Plex'] = plex
    data = data.loc[:, ['Run', 'Plex'] + list(cols)]
    
    # detect if standard has been used
    has_standard = len(data['Type'].str.extract(r"^(S[1-8])$").dropna()[0].unique()) == 8
    header['hasStandard'] = has_standard
    
    return header, data, col_df

In [ ]:
file = os.path.join(data_path, "3-Plex/20211021_3-Plex_ISA_Rawdata.csv")
h, d, c = read_raw_plate(file)
d

### read_out the computed concentrations and the expected concentration for the standards

In [ ]:
test1 = "20211021_Multiple_Data_3-Plex_Conan_ISA.xlsx"
test2 = "20211102_Multiple_Data_3-Plex_Conan_Messung 2_ISA.xlsx"
test3 = "20211102_3-Plex_ISA_Rawdata.csv"


get_run_plex(test3)

In [ ]:
def read_standard_from_conc(file):
    '''
    reads the expected start concentration for the standards
    '''
    # read_out the standard concentrations
    df = pd.read_excel(file, skiprows=7, sheet_name="Exp Conc")
    # create a Gene df from the columns
    col_df = pd.DataFrame(df.columns[2:])[0].str.extract(r"([^(]+) \(([0-9]+)\)").rename({0:"Gene", 1:"col"}, axis=1)
    col_df['S1'] = df.iloc[1:2,2:].T.reset_index().iloc[:,1].str.replace(",", ".").astype(float)
    cols = col_df.columns
    run, plex = get_run_plex(file)
    col_df['Run'] = run
    col_df['Plex'] = plex
    col_df = col_df.loc[:, ['Run', 'Plex'] + list(cols)]
    return col_df


def convert2float(df):

    for col in ['conc']:
        df.loc[:, col] = df[col].str.replace("---", "-1")
        df.loc[:, col] = df[col].str.replace(",", ".", regex=False)
        df.loc[:, col] = df[col].str.replace("OOR <", "-2", regex=False).str.replace("OOR >", "-1", regex=False)
        df.loc[:, col] = df[col].str.replace("***", "-3", regex=False).str.replace("*", "", regex=False)
        df.loc[:, col] = df[col].astype(float)
    return df

def read_conc_plate(file):
    '''
    reads from a checkimmune output excel file both the expected concentrations of the respective plex --> col_df
    and the computed values --> conc_df
    '''
    
    # read the plex info
    col_df = read_standard_from_conc(file)
    
    # read the concentration
    conc_df = pd.read_excel(file, skiprows=7, sheet_name="Obs Conc").iloc[1:, :].reset_index(drop=True)
    
    # apply the cleaned gene names to the column names
    conc_df.columns = ['Type', 'Well'] + list(col_df['Gene'])
    # keep only data columns
    conc_df = conc_df.query('Well == Well')
    conc_df = conc_df.loc[~(conc_df['Type'].str.match(r"[eE]?[SC][1-8]")), :].reset_index(drop=True)
    conc_df = conc_df.melt(id_vars=['Well', 'Type'], var_name="Gene", value_name="conc")

    # add run as id
    run, plex = get_run_plex(file)
    
    conc_df = convert2float(conc_df.set_index(['Type', 'Well'])).reset_index()
    cols = conc_df.columns
    conc_df['Run'] = run
    conc_df['Plex'] = plex
    conc_df = conc_df.loc[:, ['Run', 'Plex'] + list(cols)] 
    return conc_df, col_df

In [ ]:
file = os.path.join(data_path, "38-Plex/20211021_38-Plex_ISA_conc.xlsx")
col_df = read_standard_from_conc(file)
col_df[:3]

In [ ]:
conc_df, col_df = read_conc_plate(file)
conc_df

### csv

In [ ]:
h, data, col_df = read_raw_plate(excel_file)
col_df[:3]

In [ ]:
data

In [ ]:
def read_luminex(folder, raw_pattern="Rawdata", conc_pattern="ISA_conc"):
    '''
    reads all luminex data from one folder
    '''
    
    
    #### file reading
    # init the file lists
    raw_file_list = []
    
    conc_file_list = []
    
    for f in [folder for folder in os.walk(data_path)]:
        folder = f[0]
        raw_files = [os.path.join(folder, file) for file in f[2] if raw_pattern in file and not os.path.basename(file).startswith("~$")]
        raw_file_list += raw_files
        
        conc_files = [os.path.join(folder, file) for file in f[2] if conc_pattern in file and not os.path.basename(file).startswith("~$")]
        conc_file_list += conc_files

    # ######### raw files
    # load in all the raw_files and store data in dfs
    raw_col_dfs = []
    data_dfs = []
    plate_dfs = []
    # cycle through raw files
    for file in raw_file_list:
        print(f"Loading raw data file {file}")
        plate_df, data_df, raw_col_df = read_raw_plate(file)
        raw_col_dfs.append(raw_col_df)
        plate_dfs.append(plate_df)
        data_dfs.append(data_df)
    
    plate_df = pd.concat(plate_dfs).sort_values(['Run', 'Plex'])
    raw_col_df = pd.concat(raw_col_dfs).sort_values(['Run', 'col'])
    data_df = pd.concat(data_dfs).sort_values(['Run', 'Plex', 'Well']).reset_index(drop=True)
    
    # ######## conc_files
    conc_col_dfs = []
    conc_dfs = []

    # cycle through conc files
    for file in conc_file_list:
        print(f"Loading concentration file {file}")
        conc_df, col_df = read_conc_plate(file)
        conc_col_dfs.append(col_df)
        conc_dfs.append(conc_df)
    
    conc_col_df = pd.concat(conc_col_dfs).sort_values(['Run', 'col']).loc[:, ['Run', 'Gene', 'col', 'S1']]
    conc_df = pd.concat(conc_dfs).sort_values(['Run', 'Plex', 'Well']).reset_index(drop=True)    
    
    # check for consistency beween the plexes in raw and conc files
    col_df = raw_col_df.merge(conc_col_df, how="outer")
    
    return plate_df, col_df, data_df, conc_df

### read the data for all raw files and the "ISA_conc" files (from Sté)

In [ ]:
plate_df, col_df, data_df, conc_df = read_luminex(data_path, raw_pattern="Rawdata", conc_pattern="ISA_conc")

In [ ]:
col_df

### read the data for all raw files and the "Conan_ISA" files (from Micha)

In [ ]:
_, col_df, _, conc_df = read_luminex(data_path, raw_pattern="Rawdata", conc_pattern="Conan")

In [ ]:
conc_df.query('Run == "20211102"').query('Plex == "3-Plex"')

### combine the data

In [ ]:
cols = {}
concs = {}

patterns = ["conc","Conan"]

for conc_pattern in patterns:
    # reload the plate_df and data_df (are equal) but load conc data and derived col_df into dictionary
    plate_df, cols[conc_pattern], data_df, concs[conc_pattern] = read_luminex(data_path, raw_pattern="Rawdata", conc_pattern=conc_pattern)
    
suffix = ["_" + p for p in patterns]
col_df = cols[patterns[0]].merge(cols[patterns[1]], on=["Run", "Plex", "Gene", 'col'], how="outer", suffixes=suffix).sort_values(['Run', 'Plex', 'col'])
conc_df = concs[patterns[0]].merge(concs[patterns[1]], how="outer", on=["Run", "Plex", "Well", "Gene"], suffixes=suffix)

### combine the types

In [ ]:
conc_df.loc[conc_df['Type_Conan'].notna(), 'Type'] = conc_df['Type_Conan']
conc_df.loc[conc_df['Type_Conan'].isna(), 'Type'] = conc_df['Type_conc']
conc_df

### get the well sample connection

In [ ]:
def get_well_df(conc_df):
    '''
    retrieve the sample allocation to the wells
    '''
    # get the wells by a groupby
    well_df = conc_df.groupby(['Run', 'Well']).agg({'Type': 'first'}).reset_index()
    
    # expand the duplicates "H5,H6" into single wells
    dedup_well_df = well_df['Well'].str.extractall(r"(?P<Well>[A-H][0-9]+)").reset_index(level=1).drop("match", axis=1)
    # remerge for deduping
    well_df = dedup_well_df.merge(well_df.drop('Well', axis=1), left_index=True, right_index=True)
    return well_df.loc[:, ['Run', 'Well', 'Type']].rename({'Type': 'Sample'})

In [ ]:
well_df = get_well_df(conc_df)
well_df

### now other types can be removed from conc data

In [ ]:
conc_df = conc_df.loc[:, ['Run', 'Type', 'Well', 'Gene', 'conc_conc', 'conc_Conan']].rename({'Type': 'Sample'})
conc_df

### Type can be removed from raw data

In [ ]:
data_df.loc[~data_df['Type'].str.match(r"^[SC][1-9]?"), 'Type'] = ""
data_df = data_df.loc[:, ['Run', 'Plex', 'Well', 'Type', 'Gene', 'FI', 'SamplingErrors']]
data_df

### save to excel

In [ ]:
excel_out = os.path.join(data_path, "../output/luminexcel.xlsx")
    

with pd.ExcelWriter(excel_out, mode="w") as writer:
    plate_df.to_excel(writer, sheet_name="Plates", index=False)
    well_df.to_excel(writer, sheet_name="Wells", index=False)
    col_df.to_excel(writer, sheet_name="Plexes", index=False)
    data_df.to_excel(writer, sheet_name="RawData", index=False)
    conc_df.to_excel(writer, sheet_name="ComputedConc", index=False)